In [17]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
#from config import api_key
c = Census('d71cbb5b9686d70681b05daa33b6ea1e10dc3e40', year=2013)

In [27]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", 
                                      "state tabulation area": "state"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Name", "Population", "Poverty Rate"]]

# Visualize
print(len(census_pd))
census_pd.head()

52


,Name,Population,Poverty Rate
0,Alabama,4799277.0,18.140878
1,Alaska,720316.0,9.650487
2,Arizona,6479703.0,17.468409
3,Arkansas,2933369.0,18.658682
4,California,37659181.0,15.628107


In [28]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)

In [31]:
ff = pd.read_csv("datatoo.csv")
ff

,dateAdded,dateUpdated,address,city,country,latitude,longitude,postalCode,province,rName
0,2016-03-29T05:06:36Z,2018-06-26T02:59:52Z,206 Wears Valley Rd,Pigeon Forge,US,35.803788,-83.580553,37863,TN,Taco Bell
1,2015-10-23T23:59:49Z,2018-06-26T02:59:43Z,9768 Grand River Ave,Detroit,US,42.368823,-83.138251,48204,MI,Wendys
2,2015-09-21T07:47:08Z,2018-06-26T02:59:43Z,13600 W McNichols Rd,Detroit,US,42.416819,-83.180444,48235,MI,Burger King
3,2017-06-30T23:04:32Z,2018-06-25T09:29:45Z,1224 State St,Schenectady,US,42.797920,-73.924364,12304,NY,McDonalds
4,2017-06-29T02:03:33Z,2018-06-25T09:29:45Z,1232 Ulster Ave,Kingston,US,41.967079,-73.988583,12401,NY,McDonalds
...,...,...,...,...,...,...,...,...,...,...
4905,2016-05-23T07:37:37Z,2018-04-07T23:40:02Z,8390 N Stemmons Fwy,Dallas,US,32.826350,-96.871290,75247,TX,McDonalds
4906,2016-03-24T09:44:33Z,2018-04-07T23:40:01Z,7007 W Wheatland Rd,Dallas,US,32.647333,-96.943384,75249,TX,McDonalds
4907,2017-06-23T05:54:25Z,2018-04-07T23:39:57Z,542 Tecumseh St,Dundee,US,41.959490,-83.668100,48131,MI,McDonalds
4908,2017-06-19T14:51:08Z,2018-04-07T23:39:53Z,401 W Channel Islands Blvd,Port Hueneme,US,34.176370,-119.208850,93041,CA,McDonalds


In [67]:
# by_state = ff.groupby(['province', "rName"])['rName'].count()
# by_state.head(20)
# wendys = ff.loc[ff["rName"] == "Wendys"]
# w = wendys.groupby("province")['rName'].count()
# wendys = ff.loc[ff["rName"] == "Wendys"]
# w = wendys.groupby("province")['rName'].count()
# wendysrest = ff.loc[ff["rName"] == "Wendys"]
# w = wendys.groupby("province")['rName'].count()
# wendys = ff.loc[ff["rName"] == "Wendys"]
# w = wendys.groupby("province")['rName'].count()
# wendys = ff.loc[ff["rName"] == "Wendys"]
w = wendys.groupby("province")['rName'].count()
def restcounter(df):
    listname = list(set(df.rName))
    restdict = {}
    for x in listname:
        series = df.loc[df["rName"] == x]
        rest = series.groupby("province")['rName'].count()
        restdict[x] = rest  
    return restdict
    
dict_to_use = restcounter(ff)


In [68]:
rest_df = pd.DataFrame(dict_to_use)

In [69]:
rest_df

,Burger King,Subway,McDonalds,Wendys,Taco Bell
AK,NaN,4.0,1.0,NaN,2.0
AL,NaN,NaN,NaN,NaN,3.0
AR,3.0,10.0,16.0,12.0,14.0
AZ,18.0,18.0,56.0,14.0,40.0
CA,114.0,70.0,141.0,32.0,115.0
CO,8.0,18.0,13.0,12.0,24.0
CT,4.0,17.0,11.0,1.0,4.0
DE,1.0,3.0,5.0,1.0,6.0
FL,43.0,29.0,133.0,53.0,67.0
GA,17.0,35.0,90.0,36.0,40.0


In [70]:
abbr = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
states = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}


In [74]:
rest_df['State'] = states.values()
rest_df = rest_df.set_index("State")
rest_df

,Burger King,Subway,McDonalds,Wendys,Taco Bell
State,,,,,
Alabama,NaN,4.0,1.0,NaN,2.0
Alaska,NaN,NaN,NaN,NaN,3.0
Arizona,3.0,10.0,16.0,12.0,14.0
Arkansas,18.0,18.0,56.0,14.0,40.0
California,114.0,70.0,141.0,32.0,115.0
Colorado,8.0,18.0,13.0,12.0,24.0
Connecticut,4.0,17.0,11.0,1.0,4.0
Delaware,1.0,3.0,5.0,1.0,6.0
Florida,43.0,29.0,133.0,53.0,67.0


In [77]:
combo = pd.merge(census_pd, rest_df, left_on = "Name", right_on = "State")
combo = combo.fillna(0)
combo
combo.to_json("third.json")